# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [1]:
!python3 -m pip install -qU jq lark langchain langchain-elasticsearch langchain_openai tiktoken

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 950.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.2/895.2 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.8 MB/s eta 0:00:00


## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [15]:
pip install elasticsearch

In [38]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

Elastic Cloud ID: ··········


In [41]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

Elastic Api Key: ··········


In [40]:
# https://platform.openai.com/api-keys
OPENAI_API_KEY = getpass("OpenAI API key: ")

OpenAI API key: ··········


In [42]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# https://platform.openai.com/api-keys
OPENAI_API_KEY = getpass("OpenAI API key: ")

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="search-elhz", #give it a meaningful name,
    embedding=embeddings,
)

Elastic Cloud ID: ··········
Elastic Api Key: ··········
OpenAI API key: ··········


## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [43]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [46]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [48]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def metadata_func(record: dict, metadata: dict) -> dict:
    #Populate the metadata dictionary with keys name, summary, url, category, and updated_at.
    None

    return metadata


# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=50 #define chunk size and chunk overlap
)
docs = loader.load_and_split(text_splitter=text_splitter)

### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [50]:
documents = vectorstore.from_documents(
    docs,
    embeddings,
    index_name="search-elhz",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [53]:
pip install -U langchain langchain-community langchain-openai elasticsearch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.8/895.8 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: elasticsearch
    Found existing installation: elasticsearch 8.18.0
    Uninstalling elasticsearch-8.18.0:
      Successfully uninstalled elasticsearch-8.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-elasticsearch 0.3.2 requires elasticsearch[vectorstore-mmr]<9.0.0,>=8.13.1, but you have elasticsearch 9.0.0 which is incompatible.


In [64]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document
import logging

# Set logging level for multi_query retriever (optional)
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Prompt template for question-answering with context
LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible.

    context: {context}
    Question: "{question}"
    Answer:
    """
)

# Prompt template to format a single document with a source name
LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)

# This function makes sure every document has a 'name' in metadata
def ensure_name_metadata(docs):
    for doc in docs:
        if 'name' not in doc.metadata:
            doc.metadata['name'] = "Unknown Source"  # default name if missing
    return docs

# Combines multiple documents into a single formatted string
def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

# Build the retrieval and formatting chain
_context = RunnableParallel(
    context=retriever | ensure_name_metadata | _combine_documents,
    question=RunnablePassthrough(),
)

# Final chain: format the context into the prompt and pass to the LLM
chain = _context | LLM_CONTEXT_PROMPT | llm

# Invoke the chain with a sample question
ans = chain.invoke("what is the nasa sales team?")

# Print the result
print("---- Answer ----")
print(ans)


---- Answer ----
content='The NASA sales team refers to the North America South America sales team within the sales organization structure of the company. This team is responsible for serving customers and achieving business objectives in both North America (United States, Canada, Mexico) and South America (Central and South America). The NASA region has two Area Vice-Presidents: Laura Martinez, who is the Area Vice-President of North America, and Gary Johnson, who is the Area Vice-President of South America. \n\nThe North America South America sales team consists of dedicated account managers, sales representatives, and support staff, all led by their respective Area Vice-Presidents. Their main responsibilities include identifying and pursuing new business opportunities, nurturing existing client relationships, and ensuring customer satisfaction in the regions they cover. The team collaborates closely with other departments such as marketing, product development, and customer support 

**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?

In [65]:
# New Prompt for a Healthcare Assistant
LLM_CONTEXT_PROMPT_HEALTHCARE = ChatPromptTemplate.from_template(
    """You are a helpful healthcare assistant that answers medical FAQs. Use the context from trusted documents to answer the user's question. Be informative and mention sources when relevant.

    context: {context}
    Question: "{question}"
    Answer:
    """
)

# Create a new chain for Healthcare
healthcare_context_chain = RunnableParallel(
    context=retriever | ensure_name_metadata | _combine_documents,
    question=RunnablePassthrough(),
)

healthcare_chain = healthcare_context_chain | LLM_CONTEXT_PROMPT_HEALTHCARE | llm

# Ask a healthcare-related question
healthcare_answer = healthcare_chain.invoke("What are the early symptoms of diabetes?")
print("---- Healthcare Answer ----")
print(healthcare_answer)


---- Healthcare Answer ----
content="Early symptoms of diabetes can vary, but some common signs to look out for include increased thirst, frequent urination, unexplained weight loss, fatigue, blurred vision, and slow-healing wounds. It's important to note that these symptoms can also be indicative of other health conditions, so it's crucial to consult with a healthcare provider for proper diagnosis and treatment.\n\nIf you suspect you may have diabetes or are experiencing any of these symptoms, it's recommended to schedule an appointment with your healthcare provider for further evaluation and testing. Early detection and management of diabetes are key in preventing complications and maintaining overall health.\n\n(Source: American Diabetes Association)" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 737, 'total_tokens': 864, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_to

In [66]:
# New Prompt for Space Education
LLM_CONTEXT_PROMPT_SPACE = ChatPromptTemplate.from_template(
    """You are a space science assistant. Based on the provided context, explain the answer in a fun and educational tone. Mention the source where the information was found.

    context: {context}
    Question: "{question}"
    Answer:
    """
)

# New chain for space education
space_context_chain = RunnableParallel(
    context=retriever | ensure_name_metadata | _combine_documents,
    question=RunnablePassthrough(),
)

space_chain = space_context_chain | LLM_CONTEXT_PROMPT_SPACE | llm

# Ask a question about space
space_answer = space_chain.invoke("How does the James Webb Telescope differ from Hubble?")
print("---- Space Answer ----")
print(space_answer)


---- Space Answer ----
content="The James Webb Space Telescope and the Hubble Space Telescope are both incredible tools used to explore the universe, but they have some key differences. The Hubble Space Telescope, launched in 1990, observes the universe in visible, ultraviolet, and near-infrared light. It orbits Earth and has provided stunning images and valuable scientific data for over three decades.\n\nOn the other hand, the James Webb Space Telescope, set to launch in 2021, will observe the universe primarily in the infrared spectrum. This means it can see through dust clouds and study the earliest galaxies that formed in the universe. The James Webb Telescope will be positioned much farther from Earth than Hubble, at a location called the second Lagrange point (L2), about 1.5 million kilometers away.\n\nIn summary, while Hubble observes in visible and near-infrared light from Earth's orbit, the James Webb Telescope will focus on the infrared spectrum from a much farther distance i